In [ ]:
%matplotlib notebook

import numpy as np
import math
import pylhef
import sys
import matplotlib.pyplot as plt


In [ ]:
lhfile1 = pylhef.read('madgraph_ttbar_tWb.lhe')
#lhfile2 = pylhef.read('madgraph_ttbar_PS_tWb.lhe')
lhfile2 = pylhef.read('testout.lhe')

infiles = [lhfile1, lhfile2]

In [ ]:
Tmom = [[],[]]    #transverse momentum of top quark
mom = [[],[]]    # momentum of top quark


for i,infile in enumerate(infiles):
    for event in infile.events:      
        particles=event.particles      #finds the index of the top quark
        for particle in particles:
            if particle.id == 6:
                x = particle.p[1]
                y = particle.p[2]
                z = particle.p[3]
                momentum = math.sqrt(x**2+y**2)
                Tmom[i].append(momentum)
                momentum = math.sqrt(x**2+y**2+z**2)
                mom[i].append(momentum)
                        

#bins = np.linspace(0,1200,70) 
bins = 100
hi = 600
lo = 0

plt.figure(figsize=(10,4))
plt.subplot(1,3,1)
for T in Tmom:
    plt.hist(T, bins,range=(lo,hi), alpha=0.5, label='top momentum',histtype='step',fill=False,linewidth=4)
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of Top Quark')

#plt.figure()
plt.subplot(1,3,2)
for T in mom:
    plt.hist(T, 100,range=(lo,1200), alpha=0.5, label='top momentum',histtype='step',fill=False,linewidth=4)
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of Top Quark')

In [ ]:
Tmom_W1 = []    #transverse momentum of top quark
Tmom_W2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            z = particle.p[3]

            momentum = math.sqrt(x**2+y**2 + z**2)
            Tmom_W1.append(momentum)
            
for event in lhfile2.events:
    particles=event.particles
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            z = particle.p[3]

            momentum = math.sqrt(x**2+y**2 + z**2)
            Tmom_W2.append(momentum)

#bins = np.linspace(0,1000,70)    
bins = 100
hi = 700
lo = 0

plt.figure()
plt.hist(Tmom_W1, bins,range=(lo,hi), alpha=0.5, label='top momentum',histtype='step',fill=False,linewidth=4)
plt.hist(Tmom_W2, bins,range=(lo,hi), alpha=0.5, label='top momentum PS',histtype='step',fill=False,linewidth=4)
#plt.hist(Tmom_W1, bins, alpha=0.5, label='W momentum')
#plt.hist(Tmom_W2, bins, alpha=0.5, label='W momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of W boson')

In [ ]:
Tmom_b1 = []    #transverse momentum of top quark
Tmom_b2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_b1.append(momentum)
            
for event in lhfile2.events:
    particles=event.particles
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_b2.append(momentum)

bins = np.linspace(0,700,70) 

bins = 100
hi = 300
lo = 0

plt.figure()
plt.hist(Tmom_b1, bins,range=(lo,hi), alpha=0.5, label='top momentum',histtype='step',fill=False,linewidth=4)
plt.hist(Tmom_b2, bins,range=(lo,hi), alpha=0.5, label='top momentum PS',histtype='step',fill=False,linewidth=4)
#plt.hist(Tmom_b1, bins, alpha=0.5, label='b momentum')
#plt.hist(Tmom_b2, bins, alpha=0.5, label='b momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of bottom quark')

In [ ]:
################################################################################
def lorentz_boost(pmom, rest_frame):

    p = rest_frame
    c = 1

    pmag = np.sqrt(p[1]**2 + p[2]**2 + p[3]**2)
    #E = np.sqrt((pmag*c)**2 + (m*c**2)**2)
    E = p[0]

    beta = pmag/E
    betaX = p[1]/E
    betaY = p[2]/E
    betaZ = p[3]/E

    gamma = np.sqrt(1 / (1-beta**2))

    x = ((gamma-1) * betaX) / beta**2
    y = ((gamma-1) * betaY) / beta**2
    z = ((gamma-1) * betaZ) / beta**2

    L = np.matrix([[gamma,      -gamma*betaX, -gamma*betaY, -gamma*betaZ],
                [-gamma*betaX,  1 + x*betaX,      x*betaY,      x*betaZ],
                [-gamma*betaY,      y*betaX,  1 + y*betaY,      y*betaZ],
                [-gamma*betaZ,      z*betaX,      z*betaZ,  1 + z*betaZ]])


    # Moving particle that will be boosted
    #vector = np.matrix([E,p[1],p[1],p[2]])
    vector = np.matrix(pmom)

    boosted_vec = L*np.matrix.transpose(vector)

    return boosted_vec


In [ ]:
Tmom_b1 = []    #transverse momentum of top quark
Tmom_b2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            ptop = particle.p
            #print(ptop)
        if particle.id == 24:
            pb = particle.p
            pboost = lorentz_boost(pb,ptop)
            #print(pboost)
            x = pboost.item(1,0)
            y = pboost.item(2,0)
            z = pboost.item(3,0)
            momentum = math.sqrt(x**2+y**2+z**2)
            Tmom_b1.append(momentum)

for event in lhfile2.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            ptop = particle.p
            #print(ptop)
        if particle.id == 24:
            pb = particle.p
            pboost = lorentz_boost(pb,ptop)
            x = pboost[1]
            y = pboost[2]
            z = pboost[3]
            momentum = math.sqrt(x**2+y**2+z**2)
            Tmom_b2.append(momentum)

'''            
for event in lhfile2.events:
    particles=event.particles
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_b2.append(momentum)
'''
bins = np.linspace(0,700,70) 

bins = 100
hi = 300
lo = 0

plt.figure()
plt.hist(Tmom_b1, bins,range=(lo,hi), alpha=0.5, label='btop momentum',histtype='step',fill=False,linewidth=4)
plt.hist(Tmom_b2, bins,range=(lo,hi), alpha=0.5, label='top momentum PS',histtype='step',fill=False,linewidth=4)
#plt.hist(Tmom_b1, bins, alpha=0.5, label='b momentum')
#plt.hist(Tmom_b2, bins, alpha=0.5, label='b momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Momentum'), plt.ylabel('frequency')
plt.title('Momentum of bottom quark')

In [ ]:
def angle2vecs(a,b):
    
    #print(type(a[0]),type(b[0]))
    dp = a[0]*b[0] + a[1]*b[1] + a[2]*b[2]
    
    maga = np.sqrt(a[0]*a[0] + a[1]*a[1] +a[2]*a[2])
    magb = np.sqrt(b[0]*b[0] + b[1]*b[1] +b[2]*b[2])
    
    costh = dp/(maga*magb)
    #print(costh,dp)
    
    return costh

In [ ]:
Tmom_b1 = []    #transverse momentum of top quark
Tmom_b2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            ptop = particle.p
            #print(ptop)
        if particle.id == 24:
            pb = particle.p
            pboost = lorentz_boost(pb,ptop)
            pboost = [pboost.item(1,0),pboost.item(2,0),pboost.item(3,0) ]
            #print(pboost.item(0,0))
            costh = angle2vecs(ptop[1:],pboost)
            Tmom_b1.append(costh)


for event in lhfile2.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            ptop = particle.p
            #print(ptop)
        if particle.id == 24:
            pb = particle.p
            pboost = lorentz_boost(pb,ptop)
            pboost = [pboost.item(1,0),pboost.item(2,0),pboost.item(3,0) ]
            #print(pboost.item(0,0))
            costh = angle2vecs(ptop[1:],pboost)
            Tmom_b2.append(costh)
            
'''
for event in lhfile2.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            ptop = particle.p
            #print(ptop)
        if particle.id == 5:
            pb = particle.p
            pboost = lorentz_boost(pb,ptop)
            costh = angle2vecs(ptop[1:],pboost[1:])
            Tmom_b2.append(costh)
'''

'''
for event in lhfile2.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            ptop = particle.p
            #print(ptop)
        if particle.id == 24:
            pb = particle.p
            pboost = lorentz_boost(pb,ptop)
            x = pboost[1]
            y = pboost[2]
            z = pboost[3]
            momentum = math.sqrt(x**2+y**2+z**2)
            Tmom_b2.append(momentum)

'''            

bins = 100
hi = 1
lo = -1

plt.figure()
plt.hist(Tmom_b1, bins,range=(lo,hi), alpha=0.5, label='btop momentum',histtype='step',fill=False,linewidth=4)
plt.hist(Tmom_b2, bins,range=(lo,hi), alpha=0.5, label='top momentum PS',histtype='step',fill=False,linewidth=4)
#plt.hist(Tmom_b1, bins, alpha=0.5, label='b momentum')
#plt.hist(Tmom_b2, bins, alpha=0.5, label='b momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Momentum'), plt.ylabel('frequency')
plt.title('Momentum of bottom quark')